In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

pd.options.display.float_format = '{:,.4f}'.format

sys.path.append('../cmds/')

import TA_utils as ta

plt.style.use('ggplot')

from functools import partial
read_excel = partial(pd.read_excel, index_col=0, parse_dates=[0], io='data/momentum_data.xlsx')

factors = read_excel(sheet_name='factors (excess returns)')
mom = read_excel(sheet_name='momentum (excess returns)')

In [8]:
def performance_summary(return_data, return_data2):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*12)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(12))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    summary_stats['Corr Mkt'] = np.corr(return_data, return_data2['MKT'])[0, 1]
    summary_stats['Corr Value'] = np.corr(return_data, return_data2['HML'])[0, 1]
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [7]:
def get_sep_dict(df):
    sub_1993 = df.loc[:'1993']
    sub_2008 = df.loc['1994':'2008']
    sub_2023 = df.loc['2009':]

    df_dict={'1927-2023' : df,
            '1927-1993' : sub_1993,
            '1994-2008' : sub_2008,
            '2009-2023' : sub_2023}
    return df_dict

In [6]:
df_dict = get_sep_dict(mom)
df_fac_dict = get_sep_dict(factors)

summary_lst = []
for key in df_dict.keys():
    summary_stats = performance_summary(df_dict[key], df_fac_dict[key]).loc[:,['Mean','Volatility','Sharpe Ratio','Skewness']]
    summary_stats['Period'] = key
    summary_stats= summary_stats.reset_index().rename(columns = {'index':'Factor'}).set_index(['Period','Factor'])
    summary_lst.append(summary_stats)

factor_summary = pd.concat(summary_lst)
factor_summary

,,Mean,Volatility,Sharpe Ratio,VaR (0.05)
Period,Factor,,,,
1927-2023,UMD,0.0748,0.1631,0.4586,-0.0628
1927-1993,UMD,0.0901,0.1600,0.5632,-0.0547
1994-2008,UMD,0.1044,0.1756,0.5945,-0.0792
2009-2023,UMD,-0.0260,0.1616,-0.1611,-0.0746
